## Chapter 8 - Performances Measure

Before we start, we have to make sure that we already have our dependencies, that is 'recsys' folder

In [1]:
import os
#Check if we already have the 'recsys' folder
if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    # If not then download directly from the source
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip    
    !unzip recsys.zip

--2023-01-03 20:49:26--  https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip [following]
--2023-01-03 20:49:27--  https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15312323 (15M) [application/zip]
Saving to: ‘recsys.zip’

recsys.zip          100%[===================>]  14.60M  --.-KB/s    in 0.08s   

2023-01-03 20:49:27 (186 MB/s) - ‘recsys.zip’ sav

### Requirements
Other than the 'recsys' folder, we also have to make sure that the other required libs have already been installed<br>
```
matplotlib==3.2.2
numpy==1.19.2
pandas==1.0.5
python==3.7
scikit-learn==0.24.1
scikit-surprise==1.1.1
scipy==1.6.2
```
(If we use Google Colab, most of these libs are already installed and up-to-date, except for *scikit-surprise* which is not pre-installed by Google Colab) 


Because *scikit-surprise* is required by current notebook, we're going to install *scikit-surprise* right away

In [2]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626484 sha256=6a83221df4245477e57cba3d14e9c59c26f18e4c4de33cb236f3cd7cd16362ea
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


Import all of the required libs

In [3]:
from recsys.memories.UserToUser import UserToUser
from recsys.memories.ItemToItem import ItemToItem

from recsys.models.MatrixFactorization import MF
from recsys.models.ExplainableMF import EMF, explainable_score

from recsys.preprocessing import normalized_ratings
from recsys.preprocessing import train_test_split
from recsys.preprocessing import rating_matrix
from recsys.preprocessing import scale_ratings
from recsys.preprocessing import mean_ratings
from recsys.preprocessing import get_examples
from recsys.preprocessing import ids_encoder

from recsys.datasets import ml100k
from recsys.datasets import ml1m

from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import os

## 1. Results on MovieLens 100k Dataset

### 1.1. User-based CF

In [4]:
# Load data
ratings, movies = ml100k.load()

# Encode userid and itemid in ratings
ratings, uencoder, iencoder = ids_encoder(ratings)

# Get examples as tuples of userids and itemids and labels from raw ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# Split dataset into train set and test set
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

Download data 100.2%
Successfully downloaded ml-100k.zip 4924029 bytes.
Unzipping the ml-100k.zip zip file ...


#### 1.1.1 Evaluation with Euclidean Distance

In [5]:
# Evaluate with Euclidean distance
usertouser = UserToUser(ratings, movies, metric='euclidean')
print("==========================")
usertouser.evaluate(x_test, y_test)

Normalize users ratings ...
Initialize the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...
Evaluate the model on 10000 test data ...

MAE : 0.8125945111976461


0.8125945111976461

#### 1.1.2 Evaluation with Cosine Similarity

In [6]:
# Evaluate with cosine similarity
usertouser = UserToUser(ratings, movies, metric='cosine')
print("=========================")
usertouser.evaluate(x_test, y_test)

Normalize users ratings ...
Initialize the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...
Evaluate the model on 10000 test data ...

MAE : 0.7505910931068639


0.7505910931068639

### 1.2. Item-based CF

In [7]:
# Load data
ratings, movies = ml100k.load()

# Encode userid and itemid in ratings
ratings, uencoder, iencoder = ids_encoder(ratings)

# Get examples as tuples of userids and itemids and labels from raw ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# Split dataset into train set and test set
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

#### 1.2.1 Evaluation with Euclidean Distance

In [8]:
# Evaluation with Euclidean distance
itemtoitem = ItemToItem(ratings, movies, metric='euclidean')
print("==================")
itemtoitem.evaluate(x_test, y_test)

Normalize ratings ...
Create the similarity model ...
Compute nearest neighbors ...
Item to item recommendation model created with success ...
Evaluate the model on 10000 test data ...

MAE : 0.8277111416143341


0.8277111416143341

#### 1.2.2 Evaluation with Cosine Similarity

In [9]:
# Evaluation with cosine similarity
itemtoitem = ItemToItem(ratings, movies, metric='cosine')
print("==================")
itemtoitem.evaluate(x_test, y_test)

Normalize ratings ...
Create the similarity model ...
Compute nearest neighbors ...
Item to item recommendation model created with success ...
Evaluate the model on 10000 test data ...

MAE : 0.507794195659005


0.507794195659005

## 1.3. Matrix Factorization

In [10]:
#Define the number of epoch for training process
epochs = 10

In [11]:
# Load the ml100k dataset
ratings, movies = ml100k.load()

# Encode userid and itemid in ratings
ratings, uencoder, iencoder = ids_encoder(ratings)

m = ratings.userid.nunique()   # total number of users
n = ratings.itemid.nunique()   # total number of items

# Get examples as tuples of userids and itemids and labels from raw ratings
raw_examples, raw_labels = get_examples(ratings)

# Split dataset into train set and test set
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

# Create the model
mf = MF(m, n, k=10, alpha=0.01, lamb=1.5)

# Fit the model on the training set
history = mf.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

# Evaluate the model with testing set
print("==================")
mf.evaluate(x_test, y_test)

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/10 - loss : 2.734 - val_loss : 2.779
epoch 2/10 - loss : 1.764 - val_loss : 1.794
epoch 3/10 - loss : 1.592 - val_loss : 1.614
epoch 4/10 - loss : 1.538 - val_loss : 1.556
epoch 5/10 - loss : 1.515 - val_loss : 1.531
epoch 6/10 - loss : 1.503 - val_loss : 1.517
epoch 7/10 - loss : 1.496 - val_loss : 1.509
epoch 8/10 - loss : 1.491 - val_loss : 1.504
epoch 9/10 - loss : 1.488 - val_loss : 1.5
epoch 10/10 - loss : 1.486 - val_loss : 1.497
validation error : 1.497


1.4973507972141993

## 1.4. Non-negative Matrix Factorization

In [12]:
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Use the NMF algorithm.
nmf = NMF(n_factors=10, n_epochs=10)

# Run 5-fold cross-validation and print results.
history = cross_validate(nmf, data, measures=['MAE'], cv=5, verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Evaluating MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.9615  0.9501  0.9548  0.9582  0.9675  0.9584  0.0059  
Fit time          0.55    0.45    0.49    0.45    0.50    0.49    0.04    
Test time         0.18    0.29    0.14    0.23    0.15    0.20    0.06    


## 1.5. Explainable Matrix Factorization

In [13]:
# load data
ratings, movies = ml100k.load()

# Encode userid and itemid in ratings
ratings, uencoder, iencoder = ids_encoder(ratings)

users = sorted(ratings.userid.unique())
items = sorted(ratings.itemid.unique())

m = len(users)
n = len(items)

# Get examples as tuples of userids and itemids and labels from raw ratings
raw_examples, raw_labels = get_examples(ratings)

# Split dataset into train set and test set
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

# Create the user to user model for similarity measure
usertouser = UserToUser(ratings, movies)

# Compute explainable score
W = explainable_score(usertouser, users, items)

print("===================")
# Create the model
emf = EMF(m, n, W, alpha=0.01, beta=0.4, lamb=0.01, k=10)
# Train the model with training data
history = emf.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

print("===================")
#Evaluate model with testing data
emf.evaluate(x_test, y_test)

Normalize users ratings ...
Initialize the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...
Compute explainable scores ...
Training EMF
k=10 	 alpha=0.01 	 beta=0.4 	 lambda=0.01
epoch 1/10 - loss : 0.922 - val_loss : 1.036
epoch 2/10 - loss : 0.79 - val_loss : 0.873
epoch 3/10 - loss : 0.766 - val_loss : 0.837
epoch 4/10 - loss : 0.757 - val_loss : 0.822
epoch 5/10 - loss : 0.753 - val_loss : 0.814
epoch 6/10 - loss : 0.751 - val_loss : 0.808
epoch 7/10 - loss : 0.749 - val_loss : 0.805
epoch 8/10 - loss : 0.748 - val_loss : 0.802
epoch 9/10 - loss : 0.746 - val_loss : 0.799
epoch 10/10 - loss : 0.745 - val_loss : 0.797
MAE : 0.797


0.797347824723284

## 3. Results on MovieLens 1M (ML-1M)

### 3.1. User-based CF

In [14]:
# load ml100k ratings
ratings, movies = ml1m.load()

# Encode userid and itemid in ratings
ratings, uencoder, iencoder = ids_encoder(ratings)

# get examples as tuples of userids and itemids and labels from normalize ratings
raw_examples, raw_labels = get_examples(ratings, labels_column='rating')

# Split dataset into train set and test set
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

Download data 100.1%
Successfully downloaded ml-1m.zip 5917549 bytes.
Unzipping the ml-1m.zip zip file ...


#### 3.1.1 Evaluation with Euclidean Distance

In [15]:
# Create the user-based CF with 'euclidean' metric
usertouser = UserToUser(ratings, movies, k=20, metric='euclidean')

# Evaluate the user-based CF on the ml1m test data
print("==========================")
usertouser.evaluate(x_test, y_test)

Normalize users ratings ...
Initialize the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...
Evaluate the model on 100021 test data ...

MAE : 0.8069332535426614


0.8069332535426614

#### 3.1.2 Evaluation with Cosine Similarity

In [16]:
# Create the user-based CF with 'cosine' metric
usertouser = UserToUser(ratings, movies, k=20, metric='cosine')

# Evaluate the user-based CF on the ml1m test data
print("==========================")
usertouser.evaluate(x_test, y_test)

Normalize users ratings ...
Initialize the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...
Evaluate the model on 100021 test data ...

MAE : 0.732267005840993


0.732267005840993

### 3.2. Item-based CF

#### 3.2.1. Evaluation with Euclidean Distance

In [17]:
itemtoitem = ItemToItem(ratings, movies, metric='euclidean')
print("==========================")
#Evaluate model with testing data
itemtoitem.evaluate(x_test, y_test)

Normalize ratings ...
Create the similarity model ...
Compute nearest neighbors ...
Item to item recommendation model created with success ...
Evaluate the model on 100021 test data ...

MAE : 0.82502173206615


0.82502173206615

#### 3.2.2 Evaluation with Cosine Similarity

In [18]:
itemtoitem = ItemToItem(ratings, movies, metric='cosine')
print("==========================")
# Evaluate model with testing data
itemtoitem.evaluate(x_test, y_test)

Normalize ratings ...
Create the similarity model ...
Compute nearest neighbors ...
Item to item recommendation model created with success ...
Evaluate the model on 100021 test data ...

MAE : 0.42514728655396045


0.42514728655396045

### 3.3. Matrix Factorization

In [19]:
# Load the ml1m dataset
ratings, movies = ml1m.load()

# Encode userid and itemid in ratings
ratings, uencoder, iencoder = ids_encoder(ratings)

m = ratings.userid.nunique()   # total number of users
n = ratings.itemid.nunique()   # total number of items

# Get examples as tuples of userids and itemids and labels from raw ratings
raw_examples, raw_labels = get_examples(ratings)

# Split dataset into train set and test set
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

# Create the model
model = MF(m, n, k=10, alpha=0.01, lamb=1.5)

# Fit the model on the training set
history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

print("===================")
# Evaluate model with testing data 
model.evaluate(x_test, y_test)

Training Matrix Factorization Model ...
k=10 	 alpha=0.01 	 lambda=1.5
epoch 1/10 - loss : 1.713 - val_loss : 1.718
epoch 2/10 - loss : 1.523 - val_loss : 1.526
epoch 3/10 - loss : 1.496 - val_loss : 1.498
epoch 4/10 - loss : 1.489 - val_loss : 1.489
epoch 5/10 - loss : 1.485 - val_loss : 1.486
epoch 6/10 - loss : 1.484 - val_loss : 1.484
epoch 7/10 - loss : 1.483 - val_loss : 1.483
epoch 8/10 - loss : 1.483 - val_loss : 1.483
epoch 9/10 - loss : 1.482 - val_loss : 1.482
epoch 10/10 - loss : 1.482 - val_loss : 1.482
validation error : 1.482


1.4820034560467208

### 3.4. Non-negative Matrix Factorization

In [20]:
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-1m')

# Use the NMF algorithm.
nmf = NMF(n_factors=10, n_epochs=10)

# Run 5-fold cross-validation and print results.
history = cross_validate(nmf, data, measures=['MAE'], cv=5, verbose=True)

Dataset ml-1m could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /root/.surprise_data/ml-1m
Evaluating MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.9435  0.9456  0.9527  0.9546  0.9524  0.9498  0.0044  
Fit time          5.29    6.11    6.15    5.92    5.61    5.82    0.32    
Test time         2.15    3.63    3.68    2.57    4.07    3.22    0.73    


### 3.5. Explainable Matrix Factorization

In [21]:
# Load data
ratings, movies = ml1m.load()

# Encode userid and itemid in ratings
ratings, uencoder, iencoder = ids_encoder(ratings)

users = sorted(ratings.userid.unique())
items = sorted(ratings.itemid.unique())

m = len(users) 
n = len(items)

# Get examples as tuples of userids and itemids and labels from raw ratings
raw_examples, raw_labels = get_examples(ratings)

# Split dataset into train set and test set
(x_train, x_test), (y_train, y_test) = train_test_split(examples=raw_examples, labels=raw_labels)

# Create the user to user model for similarity measure
usertouser = UserToUser(ratings, movies)

# Compute explainable score
W = explainable_score(usertouser, users, items)

# Construct the model
emf = EMF(m, n, W, alpha=0.01, beta=0.4, lamb=0.01, k=10)
# Train the model with training set
history = emf.fit(x_train, y_train, epochs=epochs, validation_data=(x_test, y_test))

print("===================")
#Evaluate model with testing data
emf.evaluate(x_test, y_test)

Normalize users ratings ...
Initialize the similarity model ...
Compute nearest neighbors ...
User to user recommendation model created with success ...
Compute explainable scores ...
Training EMF
k=10 	 alpha=0.01 	 beta=0.4 	 lambda=0.01
epoch 1/10 - loss : 0.782 - val_loss : 0.807
epoch 2/10 - loss : 0.762 - val_loss : 0.781
epoch 3/10 - loss : 0.76 - val_loss : 0.775
epoch 4/10 - loss : 0.758 - val_loss : 0.771
epoch 5/10 - loss : 0.757 - val_loss : 0.769
epoch 6/10 - loss : 0.756 - val_loss : 0.767
epoch 7/10 - loss : 0.754 - val_loss : 0.764
epoch 8/10 - loss : 0.752 - val_loss : 0.762
epoch 9/10 - loss : 0.751 - val_loss : 0.761
epoch 10/10 - loss : 0.75 - val_loss : 0.76
MAE : 0.76


0.7596115374525224

## Summary

<center> <b> MAE comparison between User-based and Item-based CF </b> </center>

|   Metric  | Dataset | User-based | Item-based |
|:---------:|:-------:|:----------:|:----------:|
| Euclidean | ML-100k |    0.81    |    0.83    |
| Euclidean |  ML-1M  |    0.81    |    0.82    |
|   Cosine  | ML-100k |    0.75    |    0.51    |
|   Cosine  |  ML-1M  |    0.73    |    0.42    |


---


<center> <b> MAE comparison between MF, NMF and EMF </b> </center>

|  Preprocessing  | Dataset |   MF  |   NMF  | EMF   |
|:---------------:|:-------:|:-----:|:------:|-------|
|     Raw data    | ML-100k | 1.497 |  0.951 | 0.797 |
|     Raw data    |  ML-1M  | 1.482 | 0.9567 | 0.76  |
| Normalized data | ML-100k | 0.828 |   ---  | 0.783 |
| Normalized data |  ML-1M  | 0.825 |   ---  | 0.758 |


## Author

[Carmel WENGA](https://www.linkedin.com/in/carmel-wenga-871876178/), <br>
PhD student at Université de la Polynésie Française, <br> 
Applied Machine Learning Research Engineer, <br>
[ShoppingList](https://shoppinglist.cm), NzhinuSoft.